In [1]:
import os
import re
from openai import OpenAI
import ast
import json
import pandas as pd
import random
import numpy as np
import time

# Need to add before using
# OPEN_API_KEY = " "

client = OpenAI(
    api_key=OPEN_API_KEY
)

def send_openai_messages(messages, model="gpt-4o"):
    response = client.chat.completions.create(
        model= model, 
        messages=messages,
    )

    resp =response.choices[0].message.content
    return resp

def send_openai_prompt(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model= model, 
        messages=[{"role":"user", "content": prompt}],
    )

    resp =response.choices[0].message.content
    return resp

In [2]:
#send_openai_prompt("Hi, how are you?", model="gpt-4")

"As an artificial intelligence, I don't have feelings, but I'm functioning as expected. Thank you for asking! How can I assist you today?"

In [3]:
#send_openai_prompt("Hi, how are you?", model="gpt-4o")

"Hello! I'm here and ready to help you with whatever you need. How can I assist you today?"

# Basic interest

In [5]:
df_basic_interest = pd.read_excel("CABIN File.xlsx")
df_basic_interest.head()

,CABIN Data Type,Mechanics/Electronics,Construction/Woodwork,Transportation/Machine Operation,Physical/Manual Labor,Protective Service,Agriculture,Nature/Outdoors,Animal Service,Athletics,...,Sales,Marketing/Advertising,Finance,Accounting,Human Resources,Office Work,Management/Administration,Public Speaking,Politics,Law
0,Definition,Work involves maintaining and repairing of mac...,"Work involves constructing, installing, or rep...","Work involves driving vehicles, such as vans, ...",Work involves manually unloading and moving fr...,Work involves maintaining order and protecting...,"Work involves planting, cultivating, and harve...","Work involves planning, restoring, and conserv...","Work involves feeding, grooming, exercising, o...","Work involves training, practicing, or competi...",...,Work involves selling goods or services.,Work involves promoting or creating a public i...,Work involves directing and managing financial...,"Work involves examining, analyzing, and interp...","Work involves recruiting, selecting, and train...",Work involves clerical and administrative task...,"Work involves planning, directing, or coordina...",Work involves speaking before an audience to e...,Work involves engaging in political activities...,Work involves studying and applying legal know...
1,Questionnaire Items,Repair car engines.; Perform aircraft maintena...,Build wood wall shelves.; Build kitchen cabine...,Drive a bus.; Drive a delivery truck.; Operate...,Load and unload aircraft baggage.; Load and un...,Arrest suspects of criminal acts.; Conduct sur...,Farm and harvest crops.; Inspect orchards to d...,Water and fertilize garden plants.; Survey for...,Treat and care for injured animals.; Feed and ...,Play a team or individual sport.; Participate ...,...,Persuade customers to try a new product.; Incr...,Lead an advertising campaign.; Market a compan...,Make investment decisions based on financial d...,Prepare employee payroll.; Monitor account bal...,Hire employees and process hiring-related pape...,Enter personnel records into a computer progra...,Manage a medium-sized organization.; Supervise...,Present your ideas at a conference.; Speak as ...,Run for a political office.; Be the head of th...,Defend a client against a legal charge.; Prese...


In [6]:
interests = list(df_basic_interest.columns)[1:]
interests

['Mechanics/Electronics',
 'Construction/Woodwork',
 'Transportation/Machine Operation',
 'Physical/Manual Labor',
 'Protective Service',
 'Agriculture',
 'Nature/Outdoors',
 'Animal Service',
 'Athletics',
 'Engineering',
 'Physical Science',
 'Life Science',
 'Medical Science',
 'Social Science',
 'Humanities',
 'Mathematics/Statistics',
 'Information Technology',
 'Visual Arts',
 'Applied Arts and Design',
 'Performing Arts',
 'Music',
 'Writing',
 'Media',
 'Culinary Art',
 'Teaching/Education',
 'Social Service',
 'Health Care Service',
 'Religious Activities',
 'Personal Service',
 'Professional Advising',
 'Business Initiatives',
 'Sales',
 'Marketing/Advertising',
 'Finance',
 'Accounting',
 'Human Resources',
 'Office Work',
 'Management/Administration',
 'Public Speaking',
 'Politics',
 'Law']

In [7]:
def get_prompt_7(occupation, description, tasks, basic_interest, definition, questionnaire_items):
    return f"""Your job is to rate how well the basic interest “{basic_interest}” aligns with work performed by people in the occupation “{occupation}”. 

Use the following information to inform this rating.

This is a description of the occupation “{occupation}”: “{description}”. The most important tasks for the occupation “{occupation}” are: “{tasks}”.

This is the definition of the basic interest “{basic_interest}”: “{definition}”. Here are some questionnaire items used to assess the basic interest “{basic_interest}”: “{questionnaire_items}”.

Using the description and tasks associated with “{occupation}”, and the definition and questionnaire items associated with “{basic_interest}”, ask yourself: “How descriptive is the basic interest “{basic_interest}” of the work performed by the occupation “{occupation}”?”.

Rate how well the basic interest “{basic_interest}” aligns with the occupation “{occupation}” using the following 7-point scale.",
    "1 - Not at all descriptive;”
    "2 - Slightly descriptive;",
    "3 - Somewhat descriptive;",
    "4 - Moderately descriptive;",
    "5 - Quite descriptive;",
    "6 - Highly descriptive;",
    "7 - Extremely descriptive.”

Return your response with only the number.
"""

# Pilot study with example occupation

In [8]:
df_occupation = pd.read_excel("16 Occupations 6 tasks.xlsx")
df_occupation.head()

,Title,Description,Task
0,Cashiers,Receive and disburse money in establishments o...,"Receive payment by cash, check, credit cards, ..."
1,Civil Engineers,"Perform engineering duties in planning, design...","Direct engineering activities, ensuring compli..."
2,Conservation Scientists,"Manage, improve, and protect natural resources...",Apply principles of specialized fields of scie...
3,Customer Service Representatives,Interact with customers to provide basic or sc...,Confer with customers by telephone or in perso...
4,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...,"Instruct students individually and in groups, ..."


In [15]:
i = 4

occupation = df_occupation.loc[i, "Title"]
description = df_occupation.loc[i, "Description"]
tasks = df_occupation.loc[i, "Task"]

# for each of CABIN type
# for j in range(len(types)):
for j in range(21, 22):
    basic_interest = interests[j]
    definition = df_basic_interest.loc[0, basic_interest]
    questionnaire_items = df_basic_interest.loc[1, basic_interest]

    print(get_prompt_7(occupation, description, tasks, basic_interest, definition, questionnaire_items))

Your job is to rate how well the basic interest “Writing” aligns with work performed by people in the occupation “Elementary School Teachers, Except Special Education”. 

Use the following information to inform this rating.

This is a description of the occupation “Elementary School Teachers, Except Special Education”: “Teach academic and social skills to students at the elementary school level.”. The most important tasks for the occupation “Elementary School Teachers, Except Special Education” are: “Instruct students individually and in groups, using teaching methods such as lectures, discussions, and demonstrations.; Establish and enforce rules for behavior and procedures for maintaining order among the students.; Guide and counsel students with adjustment or academic problems or with special academic interests.; Adapt teaching methods and instructional materials to meet students' varying needs and interests.; Plan and conduct activities for a balanced program of instruction, demonst

# 16 Occupations 6 Tasks

In [9]:
df_occupation = pd.read_excel("16 Occupations 6 tasks.xlsx")
df_occupation.head()

,Title,Description,Task
0,Cashiers,Receive and disburse money in establishments o...,"Receive payment by cash, check, credit cards, ..."
1,Civil Engineers,"Perform engineering duties in planning, design...","Direct engineering activities, ensuring compli..."
2,Conservation Scientists,"Manage, improve, and protect natural resources...",Apply principles of specialized fields of scie...
3,Customer Service Representatives,Interact with customers to provide basic or sc...,Confer with customers by telephone or in perso...
4,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...,"Instruct students individually and in groups, ..."


In [10]:
# Create empty df
columns = ['Occupation'] + interests
occupations = list(df_occupation[["Title"]]["Title"])

data = {'Occupation': occupations}

for column in columns[1:]:
    data[column] = [None] * len(occupations)

# Create the DataFrame
df = pd.DataFrame(data)

In [12]:
# main

model = "gpt-4o"


# for i in range(len(df_occupation)):
for i in range(4, 5):
    occupation = df_occupation.loc[i, "Title"]
    description = df_occupation.loc[i, "Description"]
    tasks = df_occupation.loc[i, "Task"]

    print(occupation, end="  : ")
    # for each of CABIN type
    for j in range(len(interests)):
    # for j in range(2):
        basic_interest = interests[j]
        definition = df_basic_interest.loc[0, basic_interest]
        questionnaire_items = df_basic_interest.loc[1, basic_interest]

        prompt = get_prompt_7(occupation, description, tasks, basic_interest, definition, questionnaire_items)

        resp = send_openai_prompt(prompt, model=model)
        
        df.loc[df['Occupation'] == occupation, basic_interest] = resp
        print(".", end="")
    print()


df.to_excel(f"test_{model}.xlsx")

Elementary School Teachers, Except Special Education  : .........................................


In [13]:
model = "gpt-4"


# for i in range(len(df_occupation)):
for i in range(4, 5):
    occupation = df_occupation.loc[i, "Title"]
    description = df_occupation.loc[i, "Description"]
    tasks = df_occupation.loc[i, "Task"]

    print(occupation, end="  : ")
    # for each of CABIN type
    for j in range(len(interests)):
    # for j in range(2):
        basic_interest = interests[j]
        definition = df_basic_interest.loc[0, basic_interest]
        questionnaire_items = df_basic_interest.loc[1, basic_interest]

        prompt = get_prompt_7(occupation, description, tasks, basic_interest, definition, questionnaire_items)

        resp = send_openai_prompt(prompt, model=model)
        
        df.loc[df['Occupation'] == occupation, basic_interest] = resp
        print(".", end="")
    print()


df.to_excel(f"test_{model}.xlsx")

Elementary School Teachers, Except Special Education  : .........................................
